In [1]:
## from gurobipy import*
import networkx as nx
import pandas as pd
import numpy as np
import copy


path = '/Users/dorukeski/Desktop/Python Directory/Research/MailData.xlsx'

def readData(path_):
    df1 = pd.read_excel(path_, sheet_name ="Sheet1")
    df1 = df1[df1.node < 550]
    TH1 = df1.TH1.tolist()
    TH2 = df1.TH2.tolist()
    
    
    df2 = pd.read_excel(path_, sheet_name = "Sheet2")
    df2 = df2[df2.SOURCE < 550]
    df2 = df2[df2.TARGET < 550]
    source = df2.SOURCE.tolist()
    target = df2.TARGET.tolist()
    weight1 = df2.WEIGHT1.tolist()
    weight2 = df2.WEIGHT2.tolist()
 
    
    return df1, df2, TH1, TH2, source, target, weight1, weight2




df1, df2, T1, T2, source, target, weight1, weight2 = readData(path)

n = len(T1)

Nodes = []
for i in range(n):
    Nodes.append(i)
    
    #Parameter Initialization
W1 = []
W2 = []
T=[]
W = []
for i in range(n):
    dummy = []
    dummy = np.zeros(n)
    W1.append(dummy)
for i in range(n):
    dummy = []
    dummy = np.zeros(n)
    W2.append(dummy)
    
for i in range(len(source)):
    W1[source[i]][target[i]] = weight1[i]
    W2[source[i]][target[i]] = weight2[i]
W.append(W1)
W.append(W2)

T.append(T1)
T.append(T2)

Time = [0,1,2,3,4]


In [2]:
Time = [0,1,2,3,4]


In [3]:
from gurobipy import*

In [4]:
#gurobi model initialization
model = Model()



Y = model.addVars(Nodes,Time,vtype = GRB.BINARY,name = "Y")
X = model.addVars(Nodes,Time,vtype = GRB.BINARY,name = "X")
U = model.addVars(Nodes,vtype = GRB.BINARY,name = "U")

model.modelSense = GRB.MAXIMIZE

#cons1
expr1 = LinExpr(quicksum(U[i] for i in Nodes))
model.addConstr(expr1 <= 5 , name = "cons1")  

#cons2
model.addConstrs(Y[i,0] == U[i]   for i in Nodes)

#cons3
model.addConstrs(X[i,0] == 0 for i in Nodes)

#cons4
model.addConstrs(Y[i,t-1] <= Y[i,t] for i in Nodes for t in range(1,len(Time)))

#cons3
model.addConstrs(quicksum(W1[i][j]*X[i,t-1] for i in Nodes) + quicksum(W2[i][j]*Y[i,t-1] for i in Nodes)+U[j]
                >= T[0][j]*X[j,t]for j in Nodes for t in range(1,len(Time)))


model.addConstrs(quicksum(W1[i][j]*X[i,t-1] for i in Nodes) + quicksum(W2[i][j]*Y[i,t-1] for i in Nodes) +U[j]
                >= T[1][j]*Y[j,t]for j in Nodes for t in range(1,len(Time)))

#cons4

model.addConstrs(X[i,t] + Y[i,t] <= 1 for i in Nodes for t in Time)

#Objective Function

model.setObjective(Y.sum("*",4),GRB.MAXIMIZE)







Using license file /Users/dorukeski/gurobi.lic
Academic license - for non-commercial use only


In [5]:
#550 node results
model.optimize()



print('buying at t=0')
for i in Nodes:
    if U[i].x == 1:
        print(i)


print('buying at t=4')
for i in Nodes:
    if Y[i,4].x == 1:
        print(i)


Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 10451 rows, 6050 columns and 470500 nonzeros
Model fingerprint: 0x234a49f2
Variable types: 0 continuous, 6050 integer (6050 binary)
Coefficient statistics:
  Matrix range     [3e-04, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 6186 rows and 3503 columns
Presolve time: 4.00s
Presolved: 4265 rows, 2547 columns, 209358 nonzeros
Variable types: 0 continuous, 2547 integer (2546 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
    3070    1.8598062e+02   6.016976e+03   0.000000e+00      5s
    4970    1.2354284e+02   0.000000e+00   0.000000e+00      7s

Root relaxation: objective 1.235428e+02, 4970 iterations, 2.96 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    B

   594   568   52.72624    7 2035    7.00000   55.59683   694%   180  381s
   603   580   52.62275   11 2041    7.00000   55.59683   694%   182  387s
   615   591   52.35825   16 2022    7.00000   55.59683   694%   186  393s
   626   602   52.11262   20 1996    7.00000   55.59683   694%   189  400s
   637   616   51.79915   24 1954    7.00000   55.59683   694%   192  405s
   651   625   51.45436   29 1942    7.00000   55.59683   694%   195  411s
   660   642   51.11737   33 1919    7.00000   55.59683   694%   199  418s
   677   644   50.70485   38 1914    7.00000   55.59683   694%   201  469s
   679   662   50.55300   39 1894    7.00000   55.59683   694%   201  476s
   697   682   50.22571   45 1864    7.00000   55.59683   694%   204  485s
   717   715   49.61094   52 1844    7.00000   55.59683   694%   207  493s
   751   716   48.68506   60 2049    7.00000   55.59683   694%   206  563s
   753   717   49.96317   36 1534    7.00000   55.59683   694%   205  571s
   755   719   20.04779  

  2142  1409    9.96576  100  577    7.00000   13.60221  94.3%   300 1067s
  2169  1452   10.48102  112  496    7.00000   13.60221  94.3%   301 1071s
  2308  1483    8.79755  193  214    7.00000   13.59093  94.2%   294 1078s
  2352  1478   12.36647   66  818    7.00000   13.59093  94.2%   294 1081s
  2364  1481   11.66055   70  796    7.00000   13.59093  94.2%   297 1085s
  2397  1507   10.13542   91  676    7.00000   13.59093  94.2%   302 1091s
  2420  1525    9.23830  107  591    7.00000   13.53732  93.4%   303 1095s
  2458  1527   12.16981   67  780    7.00000   13.53732  93.4%   304 1117s
  2472  1545   11.20912   68  425    7.00000   13.53692  93.4%   305 1124s
  2513  1557   11.95557   79  764    7.00000   13.53692  93.4%   306 1130s
  2549  1564   11.18854   86  737    7.00000   13.53692  93.4%   307 1136s
  2568  1574    9.06530  100  606    7.00000   13.50629  92.9%   310 1141s
  2592  1585   11.91590   76  778    7.00000   13.50629  92.9%   314 1148s
  2613  1608   10.16216  

  7989  4429   12.03394   75  694    7.00000   12.50499  78.6%   580 2930s
  8021  4454   11.46143   83  658    7.00000   12.50499  78.6%   580 2935s
  8077  4504   10.88389   91  596    7.00000   12.50499  78.6%   578 2941s
  8123  4540    9.61855   95  553    7.00000   12.50499  78.6%   576 2945s
  8157  4542   10.40617  100  555    7.00000   12.50499  78.6%   575 2959s
  8189  4548    9.54457  103  529    7.00000   12.50499  78.6%   573 2963s
  8206  4559    9.31841  109  492    7.00000   12.50499  78.6%   573 2966s
  8247  4604    9.46934  124  455    7.00000   12.50499  78.6%   572 2972s
  8283  4626    8.39216  140  330    7.00000   12.45466  77.9%   571 2976s
  8354  4642   11.12291   77  643    7.00000   12.45466  77.9%   568 2980s
  8383  4646   10.43791   89  562    7.00000   12.45466  77.9%   567 2985s
  8453  4656     cutoff  121         7.00000   12.36815  76.7%   565 2995s
  8474  4660     cutoff   78         7.00000   12.36815  76.7%   565 3001s
  8497  4665     cutoff  

 17916  6579    8.34050   88  640    7.00000   10.54577  50.7%   574 5150s
 18040  6625    8.93139   86  659    7.00000   10.53182  50.5%   576 5194s
 18204  6637    8.86185   88  662    7.00000   10.52203  50.3%   576 5238s
 18364  6675     cutoff   82         7.00000   10.51023  50.1%   576 5282s
 18515  6680    8.62760   79  712    7.00000   10.49968  50.0%   577 5325s
 18636  6671    9.17017   77  653    7.00000   10.48968  49.9%   579 5370s
 18791  6700     cutoff   95         7.00000   10.48121  49.7%   579 5415s
 18955  6706    9.62333   90  684    7.00000   10.46217  49.5%   580 5459s
 19073  6742    9.18011   96  609    7.00000   10.45499  49.4%   581 5503s
 19239  6785    8.72157   93  590    7.00000   10.43580  49.1%   582 5548s
 19406  6815    8.48122   95  605    7.00000   10.42738  49.0%   582 5592s
 19572  6802    8.47699   99  588    7.00000   10.42024  48.9%   582 5638s
 19702  6798    9.65450   93  629    7.00000   10.40976  48.7%   583 5686s
 19855  6758    8.56220  

 24175  8081    9.06813  108  503    7.00000   10.26409  46.6%   576 7698s
 24281  8093    8.34950   96  314    7.00000   10.26409  46.6%   575 7706s
 24375  8123     cutoff  107         7.00000   10.26409  46.6%   574 7715s
 24480  8136    8.11337  115  320    7.00000   10.26409  46.6%   573 7724s
 24598  8140    9.15264   91  277    7.00000   10.25065  46.4%   572 7735s
 24691  8181    8.32534   98  494    7.00000   10.23849  46.3%   571 7743s
 24813  8209    8.92775  105  522    7.00000   10.18033  45.4%   570 7752s
 24945  8219     cutoff   95         7.00000   10.17464  45.4%   568 7765s
 25095  8206    8.32779   85  516    7.00000   10.14202  44.9%   567 7781s
 25164  8217     cutoff   84         7.00000   10.12666  44.7%   566 7804s
 25302  8246    8.23022   97  484    7.00000   10.08149  44.0%   565 7829s
 25481  8257    8.65780  113  441    7.00000   10.05100  43.6%   563 7855s
 25648  8219     cutoff  103         7.00000   10.02253  43.2%   561 7871s
 25801  8259    8.66815  

In [1]:
#250 nodes
from gurobipy import*
import networkx as nx
import pandas as pd
import numpy as np
import copy


path = '/Users/dorukeski/Desktop/Python Directory/Research/MailData.xlsx'

def readData(path_):
    df1 = pd.read_excel(path_, sheet_name ="Sheet1")
    df1 = df1[df1.node < 250]
    TH1 = df1.TH1.tolist()
    TH2 = df1.TH2.tolist()
    
    
    df2 = pd.read_excel(path_, sheet_name = "Sheet2")
    df2 = df2[df2.SOURCE < 250]
    df2 = df2[df2.TARGET < 250]
    source = df2.SOURCE.tolist()
    target = df2.TARGET.tolist()
    weight1 = df2.WEIGHT1.tolist()
    weight2 = df2.WEIGHT2.tolist()
 
    
    return df1, df2, TH1, TH2, source, target, weight1, weight2




df1, df2, T1, T2, source, target, weight1, weight2 = readData(path)

n = len(T1)

Nodes = []
for i in range(n):
    Nodes.append(i)
    
    #Parameter Initialization
W1 = []
W2 = []
T=[]
W = []
for i in range(n):
    dummy = []
    dummy = np.zeros(n)
    W1.append(dummy)
for i in range(n):
    dummy = []
    dummy = np.zeros(n)
    W2.append(dummy)
    
for i in range(len(source)):
    W1[source[i]][target[i]] = weight1[i]
    W2[source[i]][target[i]] = weight2[i]
W.append(W1)
W.append(W2)

T.append(T1)
T.append(T2)

Time = [0,1,2,3,4,5,6,7]

#gurobi model initialization
model = Model()



Y = model.addVars(Nodes,Time,vtype = GRB.BINARY,name = "Y")
X = model.addVars(Nodes,Time,vtype = GRB.BINARY,name = "X")
U = model.addVars(Nodes,vtype = GRB.BINARY,name = "U")

model.modelSense = GRB.MAXIMIZE

#cons1
expr1 = LinExpr(quicksum(U[i] for i in Nodes))
model.addConstr(expr1 <= 5 , name = "cons1")  

#cons2
model.addConstrs(Y[i,0] == U[i]   for i in Nodes)

#cons3
model.addConstrs(X[i,0] == 0 for i in Nodes)

#cons4
model.addConstrs(Y[i,t-1] <= Y[i,t] for i in Nodes for t in range(1,len(Time)))

#cons3
model.addConstrs(quicksum(W1[i][j]*X[i,t-1] for i in Nodes) + quicksum(W2[i][j]*Y[i,t-1] for i in Nodes)+U[j]
                >= T[0][j]*X[j,t]for j in Nodes for t in range(1,len(Time)))


model.addConstrs(quicksum(W1[i][j]*X[i,t-1] for i in Nodes) + quicksum(W2[i][j]*Y[i,t-1] for i in Nodes) +U[j]
                >= T[1][j]*Y[j,t]for j in Nodes for t in range(1,len(Time)))

#cons4

model.addConstrs(X[i,t] + Y[i,t] <= 1 for i in Nodes for t in Time)

#Objective Function

model.setObjective(Y.sum("*",7),GRB.MAXIMIZE)







Using license file /Users/dorukeski/gurobi.lic
Academic license - for non-commercial use only


In [2]:
#250 nodes results
model.Params.timeLimit = 10800

model.optimize()

Changed value of parameter timeLimit to 10800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 7751 rows, 4250 columns and 310004 nonzeros
Model fingerprint: 0x62f76768
Variable types: 0 continuous, 4250 integer (4250 binary)
Coefficient statistics:
  Matrix range     [6e-04, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 5194 rows and 2826 columns
Presolve time: 3.16s
Presolved: 2557 rows, 1424 columns, 114084 nonzeros
Found heuristic solution: objective 5.0000000
Variable types: 0 continuous, 1424 integer (1423 binary)

Root relaxation: objective 3.279934e+01, 3361 iterations, 0.88 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   32.79586    0 1132    5.00000  

In [3]:
print('buying at t=0')
for i in Nodes:
    if U[i].x == 1:
        print(i)


print('buying at t=7')
for i in Nodes:
    if Y[i,7].x == 1:
        print(i)


buying at t=0
14
57
150
154
200
buying at t=7
11
14
52
57
150
154
200


In [4]:
#1500 nodes
from gurobipy import*
import networkx as nx
import pandas as pd
import numpy as np
import copy


path = '/Users/dorukeski/Desktop/Python Directory/Research/MailData.xlsx'

def readData(path_):
    df1 = pd.read_excel(path_, sheet_name ="Sheet1")
    df1 = df1[df1.node < 1500]
    TH1 = df1.TH1.tolist()
    TH2 = df1.TH2.tolist()
    
    
    df2 = pd.read_excel(path_, sheet_name = "Sheet2")
    df2 = df2[df2.SOURCE < 1500]
    df2 = df2[df2.TARGET < 1500]
    source = df2.SOURCE.tolist()
    target = df2.TARGET.tolist()
    weight1 = df2.WEIGHT1.tolist()
    weight2 = df2.WEIGHT2.tolist()
 
    
    return df1, df2, TH1, TH2, source, target, weight1, weight2




df1, df2, T1, T2, source, target, weight1, weight2 = readData(path)

n = len(T1)

Nodes = []
for i in range(n):
    Nodes.append(i)
    
    #Parameter Initialization
W1 = []
W2 = []
T=[]
W = []
for i in range(n):
    dummy = []
    dummy = np.zeros(n)
    W1.append(dummy)
for i in range(n):
    dummy = []
    dummy = np.zeros(n)
    W2.append(dummy)
    
for i in range(len(source)):
    W1[source[i]][target[i]] = weight1[i]
    W2[source[i]][target[i]] = weight2[i]
W.append(W1)
W.append(W2)

T.append(T1)
T.append(T2)

Time = [0,1,2,3,4,5,6,7]

#gurobi model initialization
model = Model()



Y = model.addVars(Nodes,Time,vtype = GRB.BINARY,name = "Y")
X = model.addVars(Nodes,Time,vtype = GRB.BINARY,name = "X")
U = model.addVars(Nodes,vtype = GRB.BINARY,name = "U")

model.modelSense = GRB.MAXIMIZE

#cons1
expr1 = LinExpr(quicksum(U[i] for i in Nodes))
model.addConstr(expr1 <= 5 , name = "cons1")  

#cons2
model.addConstrs(Y[i,0] == U[i]   for i in Nodes)

#cons3
model.addConstrs(X[i,0] == 0 for i in Nodes)

#cons4
model.addConstrs(Y[i,t-1] <= Y[i,t] for i in Nodes for t in range(1,len(Time)))

#cons3
model.addConstrs(quicksum(W1[i][j]*X[i,t-1] for i in Nodes) + quicksum(W2[i][j]*Y[i,t-1] for i in Nodes)+U[j]
                >= T[0][j]*X[j,t]for j in Nodes for t in range(1,len(Time)))


model.addConstrs(quicksum(W1[i][j]*X[i,t-1] for i in Nodes) + quicksum(W2[i][j]*Y[i,t-1] for i in Nodes) +U[j]
                >= T[1][j]*Y[j,t]for j in Nodes for t in range(1,len(Time)))

#cons4

model.addConstrs(X[i,t] + Y[i,t] <= 1 for i in Nodes for t in Time)

#Objective Function

model.setObjective(Y.sum("*",7),GRB.MAXIMIZE)


In [5]:
#1500 node results
model.Params.timeLimit = 18000

model.optimize()

Changed value of parameter timeLimit to 18000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 46501 rows, 25500 columns and 2296292 nonzeros
Model fingerprint: 0x2d574b3f
Variable types: 0 continuous, 25500 integer (25500 binary)
Coefficient statistics:
  Matrix range     [3e-04, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 9826 rows and 5406 columns (presolve time = 5s) ...
Presolve removed 10378 rows and 5629 columns (presolve time = 10s) ...
Presolve removed 10738 rows and 5849 columns (presolve time = 16s) ...
Presolve removed 10887 rows and 5997 columns (presolve time = 20s) ...
Presolve removed 10965 rows and 10285 columns (presolve time = 26s) ...
Presolve removed 17900 rows and 10376 columns (presolve time = 30s) ...
Presolve removed 17900 rows and 10376 columns
Presolve time

     0     0  556.58553    0 12238   12.00000  556.58553  4538%     - 5167s
     0     0  556.28948    0 12251   12.00000  556.28948  4536%     - 5274s
     0     0  556.09926    0 12267   12.00000  556.09926  4534%     - 5357s
     0     0  555.97084    0 12267   12.00000  555.97084  4533%     - 5435s
     0     0  555.08474    0 12227   12.00000  555.08474  4526%     - 5612s
     0     0  554.57031    0 12255   12.00000  554.57031  4521%     - 5758s
     0     0  554.19192    0 12208   12.00000  554.19192  4518%     - 5873s
     0     0  553.90468    0 12241   12.00000  553.90468  4516%     - 5986s
     0     0  553.71611    0 12230   12.00000  553.71611  4514%     - 6082s
     0     0  553.58270    0 12307   12.00000  553.58270  4513%     - 6170s
     0     0  553.03043    0 12235   12.00000  553.03043  4509%     - 6333s
     0     0  552.64590    0 12377   12.00000  552.64590  4505%     - 6499s
     0     0  552.37408    0 12400   12.00000  552.37408  4503%     - 6613s
     0     0

In [7]:
print('buying at t=0')
for i in Nodes:
    if U[i].x == 1:
        print(i)


print('buying at t=7')
for i in Nodes:
    if Y[i,7].x == 1:
        print(i)


buying at t=0
13
27
154
200
353
buying at t=7
13
27
52
154
200
353
572
583
585
955
1304
1311


In [12]:
#750 nodes
from gurobipy import*
import networkx as nx
import pandas as pd
import numpy as np
import copy


path = '/Users/dorukeski/Desktop/Python Directory/Research/MailData.xlsx'

def readData(path_):
    df1 = pd.read_excel(path_, sheet_name ="Sheet1")
    df1 = df1[df1.node < 750]
    TH1 = df1.TH1.tolist()
    TH2 = df1.TH2.tolist()
    
    
    df2 = pd.read_excel(path_, sheet_name = "Sheet2")
    df2 = df2[df2.SOURCE < 750]
    df2 = df2[df2.TARGET < 750]
    source = df2.SOURCE.tolist()
    target = df2.TARGET.tolist()
    weight1 = df2.WEIGHT1.tolist()
    weight2 = df2.WEIGHT2.tolist()
 
    
    return df1, df2, TH1, TH2, source, target, weight1, weight2




df1, df2, T1, T2, source, target, weight1, weight2 = readData(path)

n = len(T1)

Nodes = []
for i in range(n):
    Nodes.append(i)
    
    #Parameter Initialization
W1 = []
W2 = []
T=[]
W = []
for i in range(n):
    dummy = []
    dummy = np.zeros(n)
    W1.append(dummy)
for i in range(n):
    dummy = []
    dummy = np.zeros(n)
    W2.append(dummy)
    
for i in range(len(source)):
    W1[source[i]][target[i]] = weight1[i]
    W2[source[i]][target[i]] = weight2[i]
W.append(W1)
W.append(W2)

T.append(T1)
T.append(T2)

Time = []
for i in range(11):
    Time.append(i)

#gurobi model initialization
model = Model()



Y = model.addVars(Nodes,Time,vtype = GRB.BINARY,name = "Y")
X = model.addVars(Nodes,Time,vtype = GRB.BINARY,name = "X")
U = model.addVars(Nodes,vtype = GRB.BINARY,name = "U")

model.modelSense = GRB.MAXIMIZE

#cons1
expr1 = LinExpr(quicksum(U[i] for i in Nodes))
model.addConstr(expr1 <= 5 , name = "cons1")  

#cons2
model.addConstrs(Y[i,0] == U[i]   for i in Nodes)

#cons3
model.addConstrs(X[i,0] == 0 for i in Nodes)

#cons4
model.addConstrs(Y[i,t-1] <= Y[i,t] for i in Nodes for t in range(1,len(Time)))

#cons3
model.addConstrs(quicksum(W1[i][j]*X[i,t-1] for i in Nodes) + quicksum(W2[i][j]*Y[i,t-1] for i in Nodes)+U[j]
                >= T[0][j]*X[j,t]for j in Nodes for t in range(1,len(Time)))


model.addConstrs(quicksum(W1[i][j]*X[i,t-1] for i in Nodes) + quicksum(W2[i][j]*Y[i,t-1] for i in Nodes) +U[j]
                >= T[1][j]*Y[j,t]for j in Nodes for t in range(1,len(Time)))

#cons4

model.addConstrs(X[i,t] + Y[i,t] <= 1 for i in Nodes for t in Time)

#Objective Function

model.setObjective(Y.sum("*",10),GRB.MAXIMIZE)


In [15]:
Time

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [13]:
#750 node results
model.Params.timeLimit = 10800

model.optimize()

Changed value of parameter timeLimit to 10800.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (mac64)
Optimize a model with 32251 rows, 17250 columns and 1632540 nonzeros
Model fingerprint: 0xb785e56c
Variable types: 0 continuous, 17250 integer (17250 binary)
Coefficient statistics:
  Matrix range     [3e-04, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective -0.0000000
Presolve removed 6816 rows and 3182 columns (presolve time = 5s) ...
Presolve removed 7352 rows and 3513 columns (presolve time = 10s) ...
Presolve removed 7752 rows and 3795 columns (presolve time = 15s) ...
Presolve removed 14744 rows and 8230 columns (presolve time = 20s) ...
Presolve removed 14746 rows and 8231 columns
Presolve time: 21.17s
Presolved: 17505 rows, 9019 columns, 986301 nonzeros
Variable types: 0 continuous, 9019 integer (9017 binary)

Root simplex log...

Iterat

     0     0  178.18710    0 7611    9.00000  178.18710  1880%     - 2531s
     0     0  178.04620    0 7618    9.00000  178.04620  1878%     - 2565s
     0     0  177.99894    0 7635    9.00000  177.99894  1878%     - 2591s
     0     0  177.37812    0 7702    9.00000  177.37812  1871%     - 2656s
     0     0  176.98489    0 7633    9.00000  176.98489  1866%     - 2713s
     0     0  176.73207    0 7615    9.00000  176.73207  1864%     - 2752s
     0     0  176.56913    0 7677    9.00000  176.56913  1862%     - 2784s
     0     0  176.44105    0 7656    9.00000  176.44105  1860%     - 2814s
     0     0  176.32121    0 7748    9.00000  176.32121  1859%     - 2850s
     0     0  176.22512    0 7654    9.00000  176.22512  1858%     - 2884s
     0     0  176.16422    0 7718    9.00000  176.16422  1857%     - 2913s
     0     0  175.85294    0 7751    9.00000  175.85294  1854%     - 2975s
     0     0  175.50167    0 7716    9.00000  175.50167  1850%     - 3035s
     0     0  175.33309  

   747   695  145.65493  190 7546   10.00000  173.92028  1639%   678 10662s
   748   696   14.44596  406 7542   10.00000  173.89177  1639%   677 10683s
   749   697  157.23736  133 7548   10.00000  173.88765  1639%   676 10693s
   750   697  156.75166  138 7551   10.00000  173.88654  1639%   675 10700s
   751   698  156.46719  140 7547   10.00000  173.88580  1639%   674 10706s

Cutting planes:
  Cover: 556
  Implied bound: 4
  Clique: 1
  MIR: 2230
  StrongCG: 89
  Flow cover: 507
  GUB cover: 708
  Zero half: 3
  RLT: 2194

Explored 751 nodes (895704 simplex iterations) in 10801.24 seconds
Thread count was 4 (of 4 available processors)

Solution count 3: 10 9 -0 

Time limit reached
Best objective 1.000000000000e+01, best bound 1.730000000000e+02, gap 1630.0000%


In [14]:
print('buying at t=0')
for i in Nodes:
    if U[i].x == 1:
        print(i)


print('buying at t=750')
for i in Nodes:
    if Y[i,750].x == 1:
        print(i)

buying at t=0
8
13
14
154
424
buying at t=750


KeyError: (0, 750)